In [1]:
# parse dependency by running setup.py, checkout per commit (when setup.py changes)
# 12f23eddde <rzhe@pku.edu.cn> - Apr 24 2021

In [1]:
import sys
import os
import pandas as pd
import numpy as np

In [2]:
# read credentials from config
import os
import configparser

config = configparser.ConfigParser()
if (os.path.exists('config.ini')):
    config.read('config.ini')
else:
    raise Exception('config.ini not found')

In [3]:
os.chdir('/data/hrz/LibraryMigrationPy')
REPOS_DIR = 'repos_50/'
if not os.path.exists(REPOS_DIR):
    os.mkdir(REPOS_DIR)
REPOS_DIR=os.path.abspath(REPOS_DIR)
REPOS_DIR

'/data/hrz/LibraryMigrationPy/repos_50'

In [4]:
from setup_runner import setup_runner, CheckoutToCommit
install_requires, out = setup_runner('repos_50/psf/requests/master/setup.py')
install_requires, out

(['chardet<5,>=3.0.2',
  'idna<3,>=2.5',
  'urllib3<1.27,>=1.21.1',
  'certifi>=2017.4.17'],
 '')

In [5]:
from pydriller import RepositoryMining
commits = RepositoryMining('repos_50/psf/requests/master/', only_in_branch='master', filepath='setup.py')
commits = list(commits.traverse_commits())
for commit in commits:
    print(commit.hash)

0477018761c67152cdcc0b83d56f27e701e65b9e
9f4ce17a29ead608a42649b3fec4a85b9911dbc9
9ad8589efd625fe125345a6e1508ee5be78fd547
9f9ebecec27108db292a31397e44d447f4ebfeb9
e1659ed7afb674006f6ce07f9908a7bccb7a8474
d511e6f148d03b5e52228b54064a24eb47c9e756
d5256802d468e997f71ba2ff3069b327cf978a5e
7229dbe0331492b16ed57745b0b448e8f26160ca
433452f9f147ce14dae0389c46a2f35ea4be9e30
d2584ff853c2cc319daa5ad5e93486f96c595d0e
b8b87d416e5e76b5aab322bd38f0b1cfb9218d01
8ffb2462f99a51dd5d2d5c4054da9e986e669899
215d3984cc72fe617a9d52e9420050fb93a3ccda
bcd30f05abfa3295a4189427c4271b6d93b98035
8521075868820f838e670df9d8fb46e249757db2
8da0abfa1a0b0de121bed615f419b4c9edaf95c8
45fabf4b9642b262e457d52660daefb160be3ad9
30d937f299d217f2107139b3c0a8083d90d184aa
37bbdbe8c3d041e5bb66396f735266c4dfeb5652
9d8d1d14f1aae1933440920953ef7841c449bef0
10eed6344cf255f48bdee0b665381969ba27bde9
31d495fb0220ff4bb812f457e03948cbe2b8e3a9
d9a9a981bc998fed82c15e5df0fd7de5fd800f40
5f768b191d76a6d582839e9ac0b2b564fdab41e9
cfb1a478acd43aea

In [6]:
from setup_runner import setup_runner, CheckoutToCommit

with CheckoutToCommit('repos_50/hall-lab/svtyper/master/', 'a82ab3f76a62d12286ba094ec7217de48636146f'):
    res = setup_runner('repos_50/hall-lab/svtyper/master/setup.py')
    print(res)

(['pysam>=0.15.0', 'numpy', 'scipy', 'cytoolz>=0.8.2'], "/home/hrz/miniconda3/lib/python3.8/site-packages/setuptools/dist.py:463: UserWarning: Normalizing 'v0.7.0' to '0.7.0'\n  warnings.warn(tmpl.format(**locals()))\n")


In [7]:
import pandas as pd
df_50 = pd.read_csv('data/projects_star_50.csv', low_memory=False)
projects_50 = df_50['Name with Owner'].to_list()
f"{len(projects_50)} projects with head {projects_50[0]}"

'28675 projects with head donnemartin/system-design-primer'

In [8]:
from pydriller import RepositoryMining
from setup_runner import setup_runner, CheckoutToCommit
import pandas as pd

def setup_parser_wrapper(project: str) -> pd.DataFrame or None:
    repo=project.split('/')[1]
    owner=project.split('/')[0]
    branch='master'
    
    # setup.py not found
    if not os.path.exists(f"{REPOS_DIR}/{owner}/{repo}/{branch}/setup.py"):
        return None
    
    # checkout to commits where setup.py changes
    df = pd.DataFrame(columns=['Name with Owner', 'commit', 'commit_time', 'deps', 'log'])
    df_idx = 0
        
    try:
        commits = RepositoryMining(f"{REPOS_DIR}/{owner}/{repo}/{branch}", only_in_branch='master', filepath='setup.py')
        commits = list(commits.traverse_commits())
        for commit in commits:
            df.at[df_idx, 'Name with Owner'] = project
            df.at[df_idx, 'commit'] = commit.hash
            df.at[df_idx, 'commit_time'] = commit.committer_date

            try:
                with CheckoutToCommit(f"{REPOS_DIR}/{owner}/{repo}/{branch}", commit.hash):
                    install_requires, out = setup_runner(f"{REPOS_DIR}/{owner}/{repo}/{branch}/setup.py")
                df.at[df_idx, 'deps'] = install_requires
                df.at[df_idx, 'log'] = out
            # catch error in checkout
            except Exception as e:
                df.at[df_idx, 'log'] = str(e)

            df_idx += 1
            
    except Exception as e:  # catch exceptions in commits parsing
        df.at[df_idx, 'Name with Owner'] = project
        df.at[df_idx, 'log'] = str(e)
        
    return df

setup_parser_wrapper('psf/requests')

,Name with Owner,commit,commit_time,deps,log
0,psf/requests,0477018761c67152cdcc0b83d56f27e701e65b9e,2011-02-13 13:52:37-05:00,None,[Errno 2] No such file or directory: 'README.r...
1,psf/requests,9f4ce17a29ead608a42649b3fec4a85b9911dbc9,2011-02-13 23:35:53-05:00,[],"Warning: 'classifiers' should be a list, got t..."
2,psf/requests,9ad8589efd625fe125345a6e1508ee5be78fd547,2011-02-14 03:58:26-05:00,[],"Warning: 'classifiers' should be a list, got t..."
3,psf/requests,9f9ebecec27108db292a31397e44d447f4ebfeb9,2011-02-14 10:35:15-05:00,[],"Warning: 'classifiers' should be a list, got t..."
4,psf/requests,e1659ed7afb674006f6ce07f9908a7bccb7a8474,2011-02-14 10:37:38-05:00,[],"Warning: 'classifiers' should be a list, got t..."
...,...,...,...,...,...
143,psf/requests,c46f55bd48dabc02f033d252f8c64e2011f37361,2020-02-18 08:58:27-06:00,"[chardet<4,>=3.0.2, idna<3,>=2.5, urllib3!=1.2...",
144,psf/requests,333fa8d3f201f68b82d9e8f26be4987e869fd3bf,2020-09-29 10:36:31-05:00,"[chardet<4,>=3.0.2, idna<3,>=2.5, urllib3!=1.2...",
145,psf/requests,03957eb1c2b9a1e5e6d61f5e930d7c5ed7cfe853,2020-11-11 11:55:50-08:00,"[chardet<4,>=3.0.2, idna<3,>=2.5, urllib3<1.27...",
146,psf/requests,516f84f490ccd352672847c6a3d23c67a13b3f98,2020-12-11 10:31:38-05:00,"[chardet<5,>=3.0.2, idna<3,>=2.5, urllib3<1.27...",


In [ ]:
from tqdm.notebook import tqdm
from utils.parallel import parallel

# run in parallel
res_df = parallel(setup_parser_wrapper, 8, projects_50, progress_bar=tqdm, return_df=True)
res_df.to_csv('data/deps_from_setup.csv', index=False)

计算进度:   0%|          | 0/28675 [00:00<?, ?it/s]

In [13]:
from tqdm.notebook import tqdm
# You may have to remove locks if the process was interrupted
def remove_lock(project: str):
    repo=project.split('/')[1]
    owner=project.split('/')[0]
    branch='master'
    
    # setup.py not found
    if not os.path.exists(f"{REPOS_DIR}/{owner}/{repo}/{branch}/setup.py"):
        return None
    
    # remove lock if exists
    if os.path.exists(f"{REPOS_DIR}/{owner}/{repo}/{branch}/.git/config.lock"):
        print(f"{REPOS_DIR}/{owner}/{repo}/{branch}/.git/config.lock")
        os.remove(f"{REPOS_DIR}/{owner}/{repo}/{branch}/.git/config.lock")
        
    # remove lock if exists
    if os.path.exists(f"{REPOS_DIR}/{owner}/{repo}/{branch}/checkout.lock"):
        print(f"{REPOS_DIR}/{owner}/{repo}/{branch}/checkout.lock")
        os.remove(f"{REPOS_DIR}/{owner}/{repo}/{branch}/checkout.lock")
        
for project in tqdm(projects_50):
    remove_lock(project)

  0%|          | 0/28675 [00:00<?, ?it/s]

/data/hrz/LibraryMigrationPy/repos_50/psf/requests/master/checkout.lock
/data/hrz/LibraryMigrationPy/repos_50/deepfakes/faceswap/master/checkout.lock
/data/hrz/LibraryMigrationPy/repos_50/getsentry/sentry/master/checkout.lock
/data/hrz/LibraryMigrationPy/repos_50/openai/gym/master/checkout.lock
/data/hrz/LibraryMigrationPy/repos_50/pypa/pipenv/master/checkout.lock
